In [ ]:
# Load the libraries needed
%matplotlib inline
import csv
import time
import numpy as np
# load machine learning libraries
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
# load plotting libraries
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Seed random number generator
np.random.seed(123)


def construct_polynomial_features(X, d):
    """
    Constructs polynomial features of a given degree for the input data.
    
    Parameters
    ----------
    X : the input data with shape (N,).
    d : the degree of the polynomial features to transform X into.
        
    Returns
    -------
    poly_features : The polynomial features [X^0, X^1, ..., X^d] with shape (N, d+1).
    """
    X = X.reshape(-1, 1) # transform the shape (N,) into shape (N, 1)
    poly_transformer = PolynomialFeatures(degree=d)  # Create PolynomialFeatures instance with the specified degree
    poly_features = poly_transformer.fit_transform(X)  # Transform the input data into polynomial features
    return poly_features


def predict_with_poly_model(X, poly_model):
    """
    Predicts the target value for given input data X using the provided polynomial model.

    Parameters:
    X: input data of shape (N,).
    poly_model: a polynomial model.

    Returns:
    Predicted target values for the given input data of shape (N,)
    """
    # Determine the degree of the polynomial from the shape of the model's coefficients
    poly_degree = poly_model.coef_.shape[-1] - 1 

    # Construct polynomial features for the input data
    poly_features = construct_polynomial_features(X, poly_degree)

    # Use the model to predict the target value for the input data
    return poly_model.predict(poly_features)


def fit_polynomial( X, y, d ):
    """
    Fits an d-degree polynomial to the data (X, y) and returns the corresponding model.

    Parameters:
    X: input data of shape (N,)
    y: target values of shape (N,)
    d: degree of the polynomial to be fit

    Returns:
    model: a LinearRegression model fitted with polynomial features of degree d
    """
    # Create a Linear Regression model without an intercept, 
    # as it's already included in the polynomial features
    model = LinearRegression(fit_intercept=False) 
    
    # Construct polynomial features
    poly_features = construct_polynomial_features(X, d)
    
    # Fit the model with the polynomial features
    model.fit(poly_features, y)
    
    return model



In [ ]:
# Demonstration of polynomial models fit to example datapoints
# You should experiment with different model degrees and different 
# numbers of sampled datapoints to get a feel for how the degree of
# the model affects the 'fit' of the model.

# Define a ground truth linear model
true_model = LinearRegression() # Initialize a LinearRegression object
true_model.coef_ = np.array([1., -4., 1.]) # Set coefficients for a 2-degree polynomial linear model
true_model.intercept_ = 0.  # Set intercept to 0 as it's already included in the polynomial features

# Generate sample data using the defined linear model with added noise
eps_std = 1.0 # Set standard deviation of noise
num_samples = 15 # Define the number of data samples to generate

X = np.random.uniform(-4., 4., size=num_samples) # Generate random sample points within the specified range
eps = np.random.normal(scale=eps_std, size=num_samples) # Generate random noise
y = predict_with_poly_model(X, true_model) + eps # Generate target values with added noises.

# Fit a polynomial model to the generated data
poly_deg = 5 # Set degree of the polynomial to fit
fitted_model = fit_polynomial(X, y, poly_deg) # Fit the polynomial to the data

# Plot the generated data
plt.figure(figsize=(4,4))
plt.scatter(X,y)
plt.ylim(y.min()-1., y.max()+1.)

# Generate sequence of x-values for plotting
axis_X = np.arange(-4.0, 4.0, 0.2).reshape(-1, 1)

# Plot the true polynomial function
poly_y_true = predict_with_poly_model(axis_X, true_model) # Calculate true y-values
plt.plot(axis_X, poly_y_true, label='True function')

# Plot the fitted polynomial function
poly_y_fitted = predict_with_poly_model(axis_X, fitted_model) # Calculate fitted y-values
plt.plot(axis_X, poly_y_fitted, label='Fitted function')

plt.title("Degree {:d} polynomial fit to example datapoints".format(poly_deg))
plt.legend() # Add legend to the plot

plt.show() # Display the plot


### <span style="color:red">=========== Assignment 1 ===========</span>

In [ ]:
# Functions for calculating MSE, bias, and variance from fitted parameters
def mean_square_error(pred_test_y_array, y):
    """
    Calculate the Mean Squared Error (MSE) of the model's predictions.

    Parameters:
    pred_test_y_array: An array of shape (num_resamplings, N) representing the predictions from 
                       different models on the test set.
    y: An array of shape (N,) representing the target values for which we are calculating the MSE.

    Returns:
    The Mean Squared Error of the model's predictions.
    """
    # Return the mean of the squared differences between the predictions and the true values
    return np.mean(np.square(pred_test_y_array - np.expand_dims(y, 0)))

def square_bias(pred_test_y_array, true_y):
    """
    Calculate the squared bias of the model's predictions.

    Parameters:
    pred_test_y_array: An array of shape (num_resamplings, N) representing the predictions from 
                       different models on the test set.
    true_y: An array of shape (N,) representing the true target values in the test set.

    Returns:
    The squared bias of the model's predictions.
    """
    # Compute the mean prediction across all models
    mean_pred_y = np.mean(pred_test_y_array, axis=0)
    
    # Return the mean of the squared difference between the mean prediction and the true values
    return np.mean(np.square(mean_pred_y - true_y))

def variance(pred_test_y_array):
    """
    Calculate the variance of the model's predictions.

    Parameters:
    pred_test_y_array: An array of shape (num_resamplings, N) representing the predictions from 
                       different models on the test set.

    Returns:
    The variance of the model's predictions.
    """
    # Return the mean of the variances of the predictions across all models
    return np.mean(np.var(pred_test_y_array, axis=0))


In [ ]:
# Number of times we sample the training dataset from the ground truth model with a fixed sample size(15).
num_resamplings = 5 

# Load the saved dataset
X, y, test_X, test_y, true_test_y = np.load("./class8_generated_data.npy", allow_pickle=True)

# Degrees of polynomials to be considered
polynomial_degrees = [2, 4, 6, 8]

# Prepare a grid of subplots for visualizing the fitted polynomials and their performance metrics.
fig, axes = plt.subplots(nrows=num_resamplings+1,
                             ncols=len(polynomial_degrees),
                             figsize=(12, 12), )

# Divide the entire data into num_resamplings subsets (each of 15 samples)
subsets_X = np.split(X, num_resamplings)
subsets_y = np.split(y, num_resamplings)

# Generate a range of x-values for plotting the fitted polynomials
axis_X = np.arange(-3.0, 3.0, 0.2)

# Iterate over each degree in the list of polynomial degrees   
for j, degree in enumerate(polynomial_degrees):
    
    # Initialize a list to store predictions on the test set from multiple models
    pred_test_y_multiple_models = []
    
    for i in range(num_resamplings):
        # Extract the ith subset of data
        subset_X = subsets_X[i] # array of shape (N_train,)
        subset_y = subsets_y[i] # array of shape (N_train,)
        
        # Plot the data points in the ith subset
        ax = axes[i, j] # select the corresponding axis
        ax.set_ylim([y.min()-1., y.max()+1.])
        ax.scatter(subset_X, subset_y)
          
        # Train a polynomial regression model on the ith subset
        fitted_model = fit_polynomial(subset_X, subset_y, degree)

        # Generate the y-values predicted by the model over the range of x-values
        pred_y = predict_with_poly_model(axis_X, fitted_model) # array of shape (N_train,)

        # Plot the fitted polynomial
        ax.plot(axis_X, pred_y)
        
        # Obtain the model's predictions on the test set
        pred_test_y = predict_with_poly_model(test_X, fitted_model) # array of shape (N_test,)
        
        # Store these predictions for later analysis
        pred_test_y_multiple_models.append(pred_test_y)
    
    # Label the top subplots with the degree of the polynomial
    axes[0, j].set_title(f"Degree {degree}")

    
    # convert the list into a numpy array of shape (num_resamplings, N_test)
    pred_test_y_array = np.array(pred_test_y_multiple_models) 

    # compute and plot the MSE, squared bias, and variance
    mse = mean_square_error(pred_test_y_array, test_y)
    bias = square_bias(pred_test_y_array, true_test_y)
    var = variance(pred_test_y_array)
    
    # Display these errors as a bar graph in the bottom subplot
    labels = ['Total Error', 'Bias^2', 'Variance']
    positions = [-2, 0, 2]
    axes[-1, j].bar(positions, [mse, bias, var])
    axes[-1, j].xaxis.set_major_locator(ticker.FixedLocator(positions))
    axes[-1, j].xaxis.set_major_formatter(ticker.FixedFormatter(labels))
polynomial_degrees = [1, 3]
    
# Show the plot
plt.show()


**Assignment 1** From the plots, describe (i) how the bias and variance relate to a degree of the polynomial and (ii)
which polynomial degree has been used to generate the data set.

<span style="color:blue">NOTE: The y-axis scales of the bar graphs are different.</span>

**Answer**: ??? (This part is to be completed by the student. Please provide your analysis and justification here.)

### <span style="color:red">=========== End of Assignment 1 ===========</span>

### <span style="color:red">=========== Assignments 2 and 3 ===========</span>

In [ ]:
# Load data
ftse_prices = []
with open('./class8_data_FTSE100.csv', 'r', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='\"')
    _ = next(csvreader) # skip the header row
    for row in csvreader:
        ftse_prices.append(float(row[1].replace(',','')) / 1e3)
ftse_prices.reverse()
y = np.array(ftse_prices) # 1e-3 * price of ftse
X = np.arange(np.size(y)) # number of months since start


# Define the polynomial degrees to consider for model fitting
polynomial_degrees = [1, 3]

# Initialize a figure with two subplots (one for each degree) in a 1x2 grid and size 10x5
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Loop over each polynomial degree
for i, degree in enumerate(polynomial_degrees):
    # Select the corresponding axis for this degree
    ax = axes[i] 

    # Plot the original data as scatter plot on the current axis
    ax.scatter(X, y)
    # Set the y-axis limits to be slightly larger than the data range for better visualization
    ax.set_ylim(y.min()-1., y.max()+1.)

    # Fit a polynomial model of the current degree to the entire training data set
    full_dataset_model = fit_polynomial(X, y, degree)
    # Predict y-values over the entire x-range using the fitted model
    full_dataset_poly_y = predict_with_poly_model(X, full_dataset_model)
    
    # Plot the fitted model prediction on the current axis
    ax.plot(X, full_dataset_poly_y, label=f"Polynomial Degree {degree}")
    
    # Label the y-axis as Price in thousands of pounds
    ax.set_ylabel("Price (£x1000)")
    # Label the x-axis as the number of months since September 2008
    ax.set_xlabel("Months since September 2008")
    # Set the title of the subplot to indicate the price and the degree of the polynomial
    ax.set_title(f"FTSE 100 index price (Degree {degree})")
    # Display a legend on the current axis
    ax.legend()

# Ensure the subplots do not overlap and display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Function to perform K-fold cross validation
def Kfold_cross_validation(X, y, degree, num_folds, verbose=False):
    """
    This function is used to perform K-fold Cross Validation. 
    It trains a polynomial model on training data and calculates the Mean Squared Error (MSE) on validation data.
    
    Parameters:
    X: The input dataset.
    y: The target values.
    degree: The degree of the polynomial we want to fit.
    num_folds: The number of folds for cross validation.
    verbose: If True, prints MSE for each fold.

    TODO: You are required to calculate the mean and standard deviation of MSE across all folds and print them.
    """
    # Record the start time of cross validation
    start_time = time.time()
    
    # Initialize the KFold object
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=123)
    kf.get_n_splits(X)
    
    # Create an empty list to store mean squared error (MSE) for each fold
    mse_losses = []
    
    # For each fold, train the model and calculate the MSE
    for i, (train_index, validation_index) in enumerate(kf.split(X)):
        # Extract training and validation data for the current fold
        train_X, validation_X = X[train_index], X[validation_index]
        train_y, validation_y = y[train_index], y[validation_index]

        # Fit the polynomial model to the training data
        fitted_model = fit_polynomial(train_X, train_y, degree)
        
        # Use the fitted model to make predictions on the validation data
        pred_y = predict_with_poly_model(validation_X, fitted_model)
        
        # Calculate MSE for the current fold and store it
        fold_mean_squared_error = np.mean(np.square(pred_y - validation_y))
        mse_losses.append(fold_mean_squared_error)
        
        # IMPORTANT: For Leave-One-Out Cross Validation (LOOCV), set 'verbose' to False. 
        # This prevents excessive output, as LOOCV produces an output for each data point.
        if verbose:
            print("Mean loss for fold {:d}/{:d}: {:.5f}".format(i+1, num_folds, fold_mean_squared_error))
    
    end_time = time.time() # Record the end time of cross validation
    elapsed_t_ms = (end_time - start_time) * 1e3
    print("Completed {:d} folds in {:.2f} milliseconds. \n".format(num_folds, elapsed_t_ms))
    
    # TODO: Calculate the mean and standard deviation of MSE across all folds and print them
    # Hint: You can use np.mean and np.std functions
    # IMPORTANT: Replace the placeholder values (-1000) with the right values.
    mean_mse_loss = -1000
    std_mse_loss = -1000
    print("Mean: {:.3f}, std: {:.3f}".format(mean_mse_loss, std_mse_loss))


# Function to perform Leave-One-Out Cross Validation (LOOCV)
def LOO_cross_validation(X, y, degree):
    """
    This function is used to perform Leave-One-Out Cross Validation (LOOCV). 
    
    LOOCV is a particular case of k-fold cross-validation where the number of folds equals the number of
    data points.

    Parameters:
    X: The input dataset.
    y: The target values.
    degree: The degree of the polynomial we want to fit.

    TODO: This function is incomplete. You are required to complement the rest of LOOCV.
          You can use the Kfold_cross_validation function as a reference.  
    """
    num_data = np.shape(X)[0]
    
    # TODO: Complement the rest of LOOCV.
    print("LOOCV - Incomplete")
    
    
# Perform cross validation with different polynomial degrees in the list 'polynomial_degrees'.
for degree in polynomial_degrees:
    print("Running 5-fold cross validation on polynomial model of degree {:d}\n".format(degree))
    Kfold_cross_validation(X, y, degree, 5, verbose=True)
    
    print(f"\n{'-' * 50}\n") # separator
    
    print("Running LOOCV on polynomial model of degree {:d}\n".format(degree))
    LOO_cross_validation(X, y, degree)
    
    print(f"\n{'-' * 50}\n") # separator 

**Assignment 2** Complement the LOOCV code. Based on the experimental results, comment on the difference between the two cross-validation methods in terms of computational efficiency.

**Answer**: ??? (This part is to be completed by yourself. Please provide your code, results and analysis.)

**Assignment 3** Calculate the average, and standard-deviation, of the mean square error loss on the validation set using LOOCV and record your results.

**Answer**: ??? (This part is to be completed based on running the LOOCV code implemented by yourself.)

### <span style="color:red">=========== End of Assignments 2 and 3 ===========</span>